## Параметрические критерии

In [1]:
import numpy as np
import pandas as pd

import scipy
from statsmodels.stats.weightstats import *
from statsmodels.stats.proportion import proportion_confint

from sklearn import cross_validation, linear_model

### Question 3

В одном из выпусков программы "Разрушители легенд" проверялось, действительно ли заразительна зевота. В эксперименте участвовало 50 испытуемых, проходивших собеседование на программу. Каждый из них разговаривал с рекрутером; в конце 34 из 50 бесед рекрутер зевал. Затем испытуемых просили подождать решения рекрутера в соседней пустой комнате.

Во время ожидания 10 из 34 испытуемых экспериментальной группы и 4 из 16 испытуемых контрольной начали зевать. Таким образом, разница в доле зевающих людей в этих двух группах составила примерно 4.4%. Ведущие заключили, что миф о заразительности зевоты подтверждён.

Можно ли утверждать, что доли зевающих в контрольной и экспериментальной группах отличаются статистически значимо? Посчитайте достигаемый уровень значимости при альтернативе заразительности зевоты, округлите до четырёх знаков после десятичной точки.

**Z-критерий для разности долей (независимые выборки)**

   | $X_1$ | $X_2$  
  ------------- | -------------|
  1  | a | b 
  0  | c | d 
  $\sum$ | $n_1$| $n_2$
  
$$ \hat{p}_1 = \frac{a}{n_1}$$

$$ \hat{p}_2 = \frac{b}{n_2}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\; \hat{p}_1 - \hat{p}_2 \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{\hat{p}_1(1 - \hat{p}_1)}{n_1} + \frac{\hat{p}_2(1 - \hat{p}_2)}{n_2}}$$

$$Z-статистика: Z({X_1, X_2}) =  \frac{\hat{p}_1 - \hat{p}_2}{\sqrt{P(1 - P)(\frac{1}{n_1} + \frac{1}{n_2})}}$$
$$P = \frac{\hat{p}_1{n_1} + \hat{p}_2{n_2}}{{n_1} + {n_2}} $$

$H_0\colon$ доли зевающих в контрольной и экспериментальной группах не отличаются

$H_1\colon$ отличаются

In [2]:
def proportions_diff_z_test(z_stat, alternative = 'two-sided'):
    if alternative not in ('two-sided', 'less', 'greater'):
        raise ValueError("alternative not recognized\n"
                         "should be 'two-sided', 'less' or 'greater'")
    
    if alternative == 'two-sided':
        return 2 * (1 - scipy.stats.norm.cdf(np.abs(z_stat)))
    
    if alternative == 'less':
        return scipy.stats.norm.cdf(z_stat)

    if alternative == 'greater':
        return 1 - scipy.stats.norm.cdf(z_stat)

In [3]:
alpha = 0.05
z = scipy.stats.norm.ppf(1 - alpha / 2.0)

a = 10.0
n1 = 34.0

b = 4.0
n2 = 16.0

p1 = a/n1
p2 = b/n2

left_boundary = (p1 - p2) - z * np.sqrt(p1 * (1 - p1)/n1 + p2 * (1 - p2)/n2)
right_boundary = (p1 - p2) + z * np.sqrt(p1 * (1 - p1)/n1 + p2 * (1 - p2)/n2)

print "95%% confidence interval for a difference between proportions: [%f, %f]" %\
      (left_boundary, right_boundary)

95% confidence interval for a difference between proportions: [-0.217558, 0.305793]


In [4]:
P = float(p1*n1 + p2*n2) / (n1 + n2)
z_stat =  (p1 - p2) / np.sqrt(P * (1 - P) * (1.0/n1 + 1.0/n2))

print "z-stat: %f" % z_stat

z-stat: 0.324102


In [5]:
print "p-value: %.4f" % proportions_diff_z_test(z_stat, alternative='greater')

p-value: 0.3729


### Question 4

Имеются данные измерений двухсот швейцарских тысячефранковых банкнот, бывших в обращении в первой половине XX века. Сто из банкнот были настоящими, и сто — поддельными.

Загрузите данные. Отделите 50 случайных наблюдений в тестовую выборку с помощью функции sklearn.cross_validation.train_test_split (зафиксируйте random state = 1). На оставшихся 150 настройте два классификатора поддельности банкнот:

* логистическая регрессия по признакам X1,X2,X3;
* логистическая регрессия по признакам X4,X5,X6.

Каждым из классификаторов сделайте предсказания меток классов на тестовой выборке. Одинаковы ли доли ошибочных предсказаний двух классификаторов? Проверьте гипотезу, вычислите достигаемый уровень значимости. Введите номер первой значащей цифры (например, если вы получили 5.5×10−8, нужно ввести 8).

**Z-критерий для разности долей (связанные выборки)**

  $X_1$ \ $X_2$ | 1| 0 | $\sum$
  ------------- | -------------|
  1  | e | f | e + f
  0  | g | h | g + h
  $\sum$ | e + g| f + h | n  
  
$$ \hat{p}_1 = \frac{e + f}{n}$$

$$ \hat{p}_2 = \frac{e + g}{n}$$

$$ \hat{p}_1 - \hat{p}_2 = \frac{f - g}{n}$$


$$\text{Доверительный интервал для }p_1 - p_2\colon \;\;  \frac{f - g}{n} \pm z_{1-\frac{\alpha}{2}}\sqrt{\frac{f + g}{n^2} - \frac{(f - g)^2}{n^3}}$$

$$Z-статистика: Z({X_1, X_2}) = \frac{f - g}{\sqrt{f + g - \frac{(f-g)^2}{n}}}$$

$H_0\colon$ доли ошибочных предсказаний одинаковы

$H_1\colon$ доли отличаются

In [6]:
data = pd.read_csv('banknotes.txt', sep='\t')
data.head()

,X1,X2,X3,X4,X5,X6,real
0,214.8,131.0,131.1,9.0,9.7,141.0,1
1,214.6,129.7,129.7,8.1,9.5,141.7,1
2,214.8,129.7,129.7,8.7,9.6,142.2,1
3,214.8,129.7,129.6,7.5,10.4,142.0,1
4,215.0,129.6,129.7,10.4,7.7,141.8,1


In [7]:
data_train, data_test, labels_train, labels_test = cross_validation.train_test_split(
    data.iloc[:, 0:6], data.loc[:, 'real'], test_size=50, random_state=1)

In [8]:
data_train1 = data_train.loc[:, ('X1', 'X2', 'X3')]
data_train2 = data_train.loc[:, ('X4', 'X5', 'X6')]

In [9]:
data_test1 = data_test.loc[:, ('X1', 'X2', 'X3')]
data_test2 = data_test.loc[:, ('X4', 'X5', 'X6')]

In [10]:
model1 = linear_model.LogisticRegression()
model1.fit(data_train1, labels_train)
model1_predictions = model1.predict(data_test1)

In [11]:
model2 = linear_model.LogisticRegression()
model2.fit(data_train2, labels_train)
model2_predictions = model2.predict(data_test2)

In [12]:
errors1 = map(int, labels_test != model1_predictions)
sum(errors1)

10

In [13]:
errors2 = map(int, labels_test != model2_predictions)
sum(errors2)

1

In [14]:
def proportions_diff_z_stat_rel(sample1, sample2):
    sample = zip(sample1, sample2)
    n = len(sample)
    
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    return float(f - g) / np.sqrt(f + g - float((f - g)**2) / n )

In [15]:
z_stat = proportions_diff_z_stat_rel(errors1, errors2)
print "z-stat: %f" % z_stat

z-stat: 2.938604


In [16]:
print "p-value: %f" % proportions_diff_z_test(z_stat, alternative='two-sided')

p-value: 0.003297


### Question 5

В предыдущей задаче посчитайте 95% доверительный интервал для разности долей ошибок двух классификаторов. Чему равна его ближайшая к нулю граница? Округлите до четырёх знаков после десятичной точки.

In [17]:
def proportions_diff_confint_rel(sample1, sample2, alpha = 0.05):
    z = scipy.stats.norm.ppf(1 - alpha / 2.)
    sample = zip(sample1, sample2)
    n = len(sample)
        
    f = sum([1 if (x[0] == 1 and x[1] == 0) else 0 for x in sample])
    g = sum([1 if (x[0] == 0 and x[1] == 1) else 0 for x in sample])
    
    left_boundary = float(f - g) / n  - z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    right_boundary = float(f - g) / n  + z * np.sqrt(float((f + g)) / n**2 - float((f - g)**2) / n**3)
    return (left_boundary, right_boundary)

In [18]:
print "95%% confidence interval for a difference between proportions: [%f, %f]" \
      % proportions_diff_confint_rel(errors1, errors2)

95% confidence interval for a difference between proportions: [0.059945, 0.300055]


### Question 6

Ежегодно более 200000 людей по всему миру сдают стандартизированный экзамен GMAT при поступлении на программы MBA. Средний результат составляет 525 баллов, стандартное отклонение — 100 баллов.

Сто студентов закончили специальные подготовительные курсы и сдали экзамен. Средний полученный ими балл — 541.4. Проверьте гипотезу о неэффективности программы против односторонней альтернативы о том, что программа работает. Отвергается ли на уровне значимости 0.05 нулевая гипотеза? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [19]:
mean = 525
sigma = 100

n = 100.0
sample_mean = 541.4

In [20]:
z_stat = (sample_mean - mean) / (sigma/np.sqrt(n))
z_stat

1.6399999999999977

In [21]:
p_value = 1 - scipy.stats.norm.cdf(z_stat)
print "p-value: %.4f" % p_value

p-value: 0.0505


### Question 7

Оцените теперь эффективность подготовительных курсов, средний балл 100 выпускников которых равен 541.5. Отвергается ли на уровне значимости 0.05 та же самая нулевая гипотеза против той же самой альтернативы? Введите достигаемый уровень значимости, округлённый до 4 знаков после десятичной точки.

In [22]:
mean = 525
sigma = 100

n = 100.0
sample_mean = 541.5

In [23]:
z_stat = (sample_mean - mean) / (sigma/np.sqrt(n))
z_stat

1.6499999999999999

In [24]:
p_value = 1 - scipy.stats.norm.cdf(z_stat)
print "p-value: %.4f" % p_value

p-value: 0.0495
